# <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2; text-align: center;">Topic Modeling</div>

An important task coming after data preprocessing and analysis (being the second step to virtually any NLP project) is topic modeling: a process that separates the existing data into multiple clusters, each of them representing a different topic. This is a crucial step in the process of understanding the data and extracting valuable insights from it. For this task, the team decided to use the BERTopic library, which is a topic modeling technique that leverages transformers model to create dense representations of the documents and then clusters them using HDBSCAN.

#### Used Embeddings

The embeddings used for topic modeling are taken from the project of digitalepidemiologylab in GitHub, which generated embeddings from COVID-19 tweets. These Embeddings are related to the BERT model, and a description about them can be found in the [official repository of digitalepidemoloylab's project](https://github.com/digitalepidemiologylab/covid-twitter-bert).

## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Dependencies Imports</div>

In [5]:
# !pip install umap-learn
# !pip install hdbscan
# !pip install sentence-transformers
# !pip install bertopic
# !pip install nltk
# !pip install nbformat
# !pip install datamapplot
# !pip install dask[dataframe]

In [2]:
import pandas as pd
import html
from umap import UMAP
from sklearn.decomposition import PCA
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import datamapplot
import re

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

c:\Users\elena\GitHub\MachineLearning-24-25-ProgressTasks\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Preprocessing</div>

Before starting to transform the data, some important preprocessing steps must be done in order to clean the data and maintain coherency in results with what has been done in the previous notebooks. The following steps were taken:

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">1. Data Loading</div>

In [3]:
DATA = 'data/'
test = pd.read_csv(DATA + 'Constraint_English_Test.csv', delimiter=';', encoding='utf-8')
train = pd.read_csv(DATA + 'Constraint_English_Train.csv', delimiter=';', encoding='utf-8')
val = pd.read_csv(DATA + 'Constraint_English_Val.csv', delimiter=';', encoding='utf-8')

tweets = pd.concat([train, val, test], ignore_index=True)
tweets.drop(columns=['id'], inplace=True)
tweets['tweet'] = tweets['tweet'].apply(html.unescape)


### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">2. Data Cleaning</div>

In [4]:
processed_tweets = tweets.copy()
processed_tweets = processed_tweets.drop_duplicates(subset='tweet', keep='first')

# remove links
def filter(tweet:str):
    # https://stackoverflow.com/questions/3809401/what-is-a-good-regular-expression-to-match-a-url#3809435
    tweet = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', '', tweet)
    tweet = re.sub(r'\d+', '', tweet) # remove numbers, not useful for topic modeling
    return tweet

processed_tweets['tweet_nolinks'] = processed_tweets['tweet'].apply(filter)

# Recalculate indexes
processed_tweets['tweet'].reset_index(drop=True, inplace=True)

processed_tweets

,tweet,label,tweet_nolinks
0,The CDC currently reports 99031 deaths. In gen...,real,The CDC currently reports deaths. In general ...
1,States reported 1121 deaths a small rise from ...,real,States reported deaths a small rise from last...
2,Politically Correct Woman (Almost) Uses Pandem...,fake,Politically Correct Woman (Almost) Uses Pandem...
3,#IndiaFightsCorona: We have 1524 #COVID testin...,real,#IndiaFightsCorona: We have #COVID testing la...
4,Populous states can generate large case counts...,real,Populous states can generate large case counts...
...,...,...,...
10695,#CoronaVirusUpdates: State-wise details of Tot...,real,#CoronaVirusUpdates: State-wise details of Tot...
10696,Tonight 12(midnight) onwards Disaster Manageme...,fake,Tonight (midnight) onwards Disaster Management...
10697,296 new cases of #COVID19Nigeria; Plateau-85 E...,real,new cases of #COVIDNigeria; Plateau- Enugu- O...
10698,RT @CDCemergency: #DYK? @CDCgov’s One-Stop Sho...,real,RT @CDCemergency: #DYK? @CDCgov’s One-Stop Sho...


## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Data Transformation (Embeddings)</div>

In order to extract the different topics, a set of pre-trained models, called embeddings, can be used to transform the data into a format that can be used by the topic modeling algorithm. The adecuacy of the embeddings to the domain of the data is paramount for the well-performing of the topic modeling algorithm. In this case, the embeddigs used will be `covid-twitter-bert`, a model provided by digitalepidemiologylab.

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">1. Load Embeddings</div>

In [7]:
from sentence_transformers import SentenceTransformer
# Load the pre-trained model
sentence_model = SentenceTransformer('digitalepidemiologylab/covid-twitter-bert-v2')

# Encode the tweets
embeddings_real = sentence_model.encode(processed_tweets[processed_tweets["label"] == "real"]["tweet_nolinks"].reset_index(drop=True).tolist(), show_progress_bar=True, batch_size=16)
embeddings_fake = sentence_model.encode(processed_tweets[processed_tweets["label"] == "fake"]["tweet_nolinks"].reset_index(drop=True).tolist(), show_progress_bar=True, batch_size=16)

No sentence-transformers model found with name digitalepidemiologylab/covid-twitter-bert-v2. Creating a new one with mean pooling.
Batches: 100%|██████████| 319/319 [10:39<00:00,  2.01s/it]


In [8]:
# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=10, metric='cosine', random_state=42, low_memory=False)

In [9]:
# Step 3 - Cluster reduced embeddings
cluster_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [10]:
# Step 4 - bag-of-words representation
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1,2))

In [11]:
# Step 3 - Cluster reduced embeddings
cluster_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [12]:
# Step 5 - Create topic representation (extract topic words)
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

In [14]:
# Step 6 - (Optional) Fine-tune topic representations with a `bertopic.representation` model
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
representation_model = {
    "KeyBERT": KeyBERTInspired(),
    # "OpenAI": openai_model,  # Uncomment if you will use OpenAI, or other LLM
    "MMR": MaximalMarginalRelevance(),
}

In [15]:
# All steps together
topic_model_real = BERTopic(
  embedding_model=sentence_model,          # Step 1 - Extract embeddings
  umap_model=umap_model,                     # Step 2 - Reduce dimensionality
  hdbscan_model=cluster_model,              # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model, # Step 6 - (Optional) Fine-tune topic representations
  calculate_probabilities = True,
  min_topic_size = 50,
  n_gram_range=(1, 2),
  verbose = True,
  language='english'
)

umap_model_fake = UMAP(n_neighbors=min(len([tweet for tweet in processed_tweets[processed_tweets["label"] == "fake"]["tweet"]]) - 1, 15), n_components=10, metric='cosine', random_state=None, low_memory=False)
topic_model_fake = BERTopic(
  embedding_model=sentence_model,          # Step 1 - Extract embeddings
  umap_model=umap_model_fake,                     # Step 2 - Reduce dimensionality
  hdbscan_model=cluster_model,              # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model, # Step 6 - (Optional) Fine-tune topic representations
  calculate_probabilities = True,
  min_topic_size = 50,
  n_gram_range=(1, 2),
  verbose = True,
  language='english'
)

In [16]:
# Trainning process
topics_real, probs_real = topic_model_real.fit_transform([tweet for tweet in processed_tweets[processed_tweets['label'] == 'real']['tweet_nolinks']], embeddings_real)

2025-01-10 22:33:27,363 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-01-10 22:34:00,646 - BERTopic - Dimensionality - Completed ✓
2025-01-10 22:34:00,656 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-01-10 22:34:01,981 - BERTopic - Cluster - Completed ✓
2025-01-10 22:34:01,994 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-01-10 22:39:13,598 - BERTopic - Representation - Completed ✓


In [17]:
topics_fake, probs_fake = topic_model_fake.fit_transform([tweet for tweet in processed_tweets[processed_tweets["label"] == "fake"]["tweet_nolinks"]], embeddings_fake)

2025-01-10 22:39:13,936 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-01-10 22:39:18,806 - BERTopic - Dimensionality - Completed ✓
2025-01-10 22:39:18,822 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-01-10 22:39:19,742 - BERTopic - Cluster - Completed ✓
2025-01-10 22:39:19,757 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-01-10 22:44:41,328 - BERTopic - Representation - Completed ✓


#### Results REAL

In [22]:
topic_model_real.get_topic(1, full=True) #all form of topic 1 representation

{'Main': [('says', 0.28101301067670764),
  ('restrictions', 0.26306302985926966),
  ('uk', 0.25833728015925606),
  ('coronavirus', 0.24886199534899672),
  ('england', 0.2451162695064336),
  ('government', 0.23143647913167364),
  ('boris', 0.23133687610331377),
  ('boris johnson', 0.23133687610331377),
  ('kayburley', 0.23133687610331377),
  ('johnson', 0.23038038414223347)],
 'KeyBERT': [('keir starmer', 0.4205571),
  ('sir keir', 0.36757615),
  ('starmer', 0.33049646),
  ('keir', 0.28094074),
  ('kayburley', 0.27781287),
  ('coronavirus uk', 0.22847211),
  ('sky news', 0.22543813),
  ('nicola sturgeon', 0.22314234),
  ('boris johnson', 0.20740908),
  ('sturgeon', 0.19335864)],
 'MMR': [('government', 0.23143647913167364),
  ('boris', 0.23133687610331377),
  ('boris johnson', 0.23133687610331377),
  ('kayburley', 0.23133687610331377),
  ('lockdown', 0.21980751558253567),
  ('follow live', 0.1926533234364441),
  ('latest coronavirus', 0.19104496490028908),
  ('health secretary', 0.19104

In [33]:
#custom labels (optional)
#main: key words, general terms
#keybert: concrete terms, maybe too specific and descriptive
#mmr: balance relevance and diversity, might be complex to understand
keybert_topic_labels = {topic: " | ".join(list(zip(*values))[0][:3]) for topic, values in topic_model_real.topic_aspects_["MMR"].items()}
topic_model_real.set_topic_labels(keybert_topic_labels)

In [34]:
# Topics visualization
topic_model_real.visualize_topics(custom_labels=True)

In [35]:
# hierarchical topic visualization
topic_model_real.visualize_hierarchy(custom_labels=True) #false for default labels

In [40]:
# visualize documents
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings_real)

# Visualize the documents in 2-dimensional space and show the titles on hover instead of the abstracts
# NOTE: You can hide the hover with `hide_document_hover=True` which is especially helpful if you have a large dataset
topic_model_real.visualize_documents(tweets['tweet'], reduced_embeddings=reduced_embeddings, custom_labels=True, hide_annotations=True, hide_document_hover=True)

In [52]:
topic_model_real.visualize_heatmap(n_clusters=20)

In [47]:
topic_model_real.visualize_barchart(top_n_topics=8)

#### Results FAKE

In [36]:
topic_model_fake.visualize_topics()

In [38]:
topic_model_fake.visualize_hierarchy()